In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import time

class SudokuGUI:
    def __init__(self, master):
        self.master = master
        master.title("Sudoku")
        self.master.resizable(False, False)  # Prevent resizing of the window
        self.board = [[tk.StringVar() for _ in range(9)] for _ in range(9)]
        self.cells = [[None for _ in range(9)] for _ in range(9)]
        self.start_time = None

        # Styling
        self.style = ttk.Style()
        self.style.configure("TButton", font=('Helvetica', 12), padding=6)
        self.style.configure("TEntry", font=('Helvetica', 14), padding=6)
        self.style.configure("TLabel", font=('Helvetica', 16))

        # Sudoku Grid Frame
        self.grid_frame = ttk.Frame(master)
        self.grid_frame.grid(row=0, column=0, padx=10, pady=5)

        # Create grid of cells
        for i in range(9):
            for j in range(9):
                bg_color = '#f0f0f0' if (i // 3 + j // 3) % 2 == 0 else '#cccccc'
                cell = ttk.Entry(self.grid_frame, width=2, font=('Arial', 18), justify='center', textvariable=self.board[i][j], background=bg_color)
                cell.grid(row=i, column=j, stick="nsew", padx=1, pady=1)
                self.cells[i][j] = cell

                # Add borders to create 3x3 grid layout
                if i % 3 == 0:
                    cell.grid(pady=(3, 1))  # Add top border
                if j % 3 == 0:
                    cell.grid(padx=(3, 1))  # Add left border
                if i == 8:
                    cell.grid(pady=(1, 3))  # Add bottom border
                if j == 8:
                    cell.grid(padx=(1, 3))  # Add right border

        # Level Selection and Puzzle Selection
        self.level_var = tk.StringVar(value="Easy")
        level_frame = ttk.Frame(master)
        level_frame.grid(row=0, column=1, padx=10, pady=5, sticky="n")
        ttk.Label(level_frame, text="Level:").grid(row=0, column=0, padx=5, sticky="e")
        self.level_menu = ttk.OptionMenu(level_frame, self.level_var, "Easy", "Easy", "Medium", "Hard", command=self.change_level)
        self.level_menu.grid(row=0, column=1, padx=5, sticky="ew")

        # Puzzle Selection
        self.puzzle_var = tk.StringVar(value="Puzzle 1")
        self.puzzle_frame = ttk.Frame(master)
        self.puzzle_frame.grid(row=1, column=1, padx=10, pady=5, sticky="n")
        ttk.Label(self.puzzle_frame, text="Puzzle:").grid(row=0, column=0, padx=5, sticky="e")
        puzzle_options = ["Puzzle 1", "Puzzle 2", "Puzzle 3", "Puzzle 4"]
        for i, option in enumerate(puzzle_options):
            ttk.Radiobutton(self.puzzle_frame, text=option, variable=self.puzzle_var, value=option, command=self.load_puzzle).grid(row=i+1, column=0, padx=5, sticky="w")

        # Timer Label
        self.timer_label = ttk.Label(master, text="Time: 0.000 seconds")
        self.timer_label.grid(row=2, column=1, padx=10, pady=(5, 10), sticky="n")

        # Solve and Reset Buttons
        solve_frame = ttk.Frame(master)
        solve_frame.grid(row=0, column=2, rowspan=3, padx=10, pady=5, sticky="ns")

        ttk.Button(solve_frame, text="Solve (Backtracking)", command=self.solve_backtracking).pack(fill="x", padx=5, pady=5)
        ttk.Button(solve_frame, text="Solve (Arc-3)", command=self.solve_arc3).pack(fill="x", padx=5, pady=5)
        ttk.Button(solve_frame, text="Reset", command=self.reset).pack(fill="x", padx=5, pady=5)

        # Load initial puzzle
        self.load_puzzle()

    def change_level(self, level):
        self.load_puzzle()

    def load_puzzle(self):
        self.reset()
        level = self.level_var.get()
        puzzle_number = int(self.puzzle_var.get().split()[-1]) - 1  # Convert puzzle number to index
        puzzles = {
            "Easy": [
                [
                    [0, 7, 0, 3, 5, 0, 8, 0, 0],
                    [0, 3, 8, 7, 1, 4, 0, 6, 9],
                    [6, 4, 5, 0, 0, 0, 7, 1, 3],
                    [5, 8, 0, 1, 0, 0, 4, 0, 0],
                    [0, 0, 2, 0, 0, 9, 3, 0, 7],
                    [3, 9, 0, 4, 7, 8, 2, 5, 1],
                    [9, 5, 0, 2, 4, 1, 0, 0, 0],
                    [0, 6, 0, 8, 9, 5, 1, 0, 2],
                    [8, 2, 1, 6, 3, 7, 0, 0, 5]
                ],
                [
                    [0, 7, 1, 0, 8, 0, 0, 3, 0],
                    [3, 0, 9, 6, 5, 7, 0, 1, 0],
                    [0, 0, 2, 0, 1, 9, 6, 8, 0],
                    [0, 6, 0, 2, 3, 5, 8, 0, 1],
                    [0, 2, 3, 0, 9, 0, 7, 5, 0],
                    [1, 0, 8, 0, 0, 6, 3, 9, 2],
                    [8, 1, 6, 0, 0, 3, 0, 2, 5],
                    [7, 3, 5, 0, 0, 4, 9, 0, 0],
                    [2, 9, 4, 5, 0, 8, 1, 7, 3]
                ],
                [
                    [7, 6, 1, 9, 3, 4, 0, 0, 2],
                    [5, 0, 0, 8, 2, 1, 0, 6, 0],
                    [0, 0, 2, 6, 7, 0, 0, 1, 4],
                    [0, 1, 0, 3, 0, 6, 0, 0, 0],
                    [9, 3, 0, 0, 0, 7, 0, 8, 5],
                    [0, 5, 7, 2, 8, 9, 0, 3, 0],
                    [0, 2, 9, 5, 0, 8, 4, 0, 0],
                    [0, 0, 5, 7, 9, 3, 0, 2, 6],
                    [6, 7, 3, 4, 0, 2, 0, 9, 0]
                ],
                [
                    [1, 0, 0, 0, 0, 0, 3, 5, 6],
                    [9, 0, 0, 4, 3, 5, 1, 2, 0],
                    [0, 0, 0, 1, 2, 0, 0, 0, 4],
                    [8, 2, 7, 3, 6, 0, 0, 0, 1],
                    [0, 0, 1, 0, 5, 0, 0, 6, 0],
                    [0, 3, 6, 0, 1, 9, 8, 4, 0],
                    [0, 5, 4, 2, 9, 3, 6, 0, 8],
                    [0, 0, 9, 6, 8, 7, 4, 3, 0],
                    [6, 8, 3, 5, 4, 0, 2, 7, 9]
                ]
            ],
            "Medium": [
                [
                    [2, 7, 3, 0, 0, 0, 0, 8, 5],
                    [0, 0, 1, 8, 0, 0, 0, 7, 0],
                    [5, 0, 0, 0, 0, 0, 0, 0, 1],
                    [0, 0, 0, 0, 8, 9, 0, 4, 0],
                    [0, 0, 8, 0, 6, 5, 0, 3, 7],
                    [4, 0, 7, 0, 0, 2, 8, 5, 0],
                    [3, 5, 0, 1, 7, 0, 0, 2, 4],
                    [0, 0, 0, 0, 0, 0, 7, 1, 0],
                    [7, 0, 0, 9, 0, 3, 0, 6, 8]
                ],
                [
                    [3, 5, 6, 0, 8, 0, 9, 0, 1],
                    [2, 0, 0, 1, 0, 3, 7, 5, 6],
                    [9, 1, 7, 0, 0, 0, 0, 0, 8],
                    [4, 8, 0, 6, 0, 0, 3, 0, 0],
                    [0, 0, 0, 8, 3, 0, 1, 0, 0],
                    [0, 3, 0, 0, 0, 2, 6, 8, 5],
                    [5, 6, 4, 0, 0, 9, 8, 1, 0],
                    [0, 0, 0, 0, 1, 0, 0, 0, 0],
                    [0, 0, 0, 4, 0, 0, 0, 0, 7]
                ],
                [
                    [9, 6, 2, 0, 4, 0, 0, 7, 0],
                    [7, 0, 0, 1, 0, 0, 0, 2, 0],
                    [3, 5, 1, 8, 0, 2, 9, 6, 4],
                    [0, 0, 3, 7, 8, 4, 0, 1, 0],
                    [8, 1, 0, 0, 0, 6, 0, 0, 0],
                    [0, 0, 7, 0, 0, 5, 0, 8, 2],
                    [0, 0, 0, 9, 0, 0, 0, 4, 0],
                    [4, 0, 0, 0, 0, 0, 0, 0, 5],
                    [0, 7, 5, 0, 0, 0, 0, 0, 0]
                ],
                [
                    [5, 0, 0, 0, 0, 0, 7, 8, 0],
                    [0, 0, 0, 8, 0, 0, 0, 1, 0],
                    [0, 3, 7, 0, 1, 0, 9, 0, 0],
                    [0, 0, 0, 1, 0, 0, 6, 0, 8],
                    [9, 8, 0, 3, 2, 6, 0, 0, 0],
                    [0, 0, 3, 0, 0, 0, 0, 9, 0],
                    [3, 0, 0, 0, 6, 4, 0, 5, 9],
                    [0, 7, 9, 2, 5, 0, 3, 4, 0],
                    [6, 0, 4, 9, 3, 0, 0, 3, 0]
                ]
            ],
            "Hard": [
                [
                    [7, 0, 0, 0, 0, 5, 0, 0, 0],
                    [0, 0, 0, 1, 0, 4, 6, 5, 0],
                    [0, 0, 0, 0, 6, 0, 3, 0, 1],
                    [3, 0, 6, 4, 0, 0, 0, 0, 0],
                    [0, 0, 4, 8, 0, 0, 1, 0, 9],
                    [9, 0, 0, 0, 7, 0, 4, 6, 0],
                    [0, 0, 0, 0, 0, 3, 0, 0, 2],
                    [5, 0, 0, 0, 0, 0, 9, 0, 0],
                    [1, 0, 8, 0, 4, 0, 5, 0, 0]
                ],
                [
                    [0, 0, 9, 0, 0, 0, 3, 0, 0],
                    [0, 0, 1, 0, 0, 4, 9, 6, 0],
                    [0, 6, 0, 0, 8, 0, 0, 0, 4],
                    [8, 0, 6, 4, 5, 3, 0, 9, 0],
                    [0, 0, 0, 0, 0, 0, 0, 8, 0],
                    [5, 0, 0, 0, 6, 0, 0, 3, 0],
                    [6, 0, 2, 0, 4, 0, 0, 0, 0],
                    [0, 1, 3, 5, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 2, 0, 0, 0, 0]
                ],
                [
                    [8, 0, 0, 0, 0, 0, 0, 0, 3],
                    [0, 0, 3, 0, 0, 4, 7, 0, 0],
                    [0, 0, 2, 0, 0, 0, 0, 0, 0],
                    [0, 0, 1, 5, 8, 0, 9, 7, 0],
                    [3, 0, 0, 0, 0, 0, 8, 5, 1],
                    [0, 0, 5, 0, 0, 0, 0, 0, 4],
                    [9, 0, 0, 0, 0, 7, 0, 0, 0],
                    [0, 0, 0, 4, 0, 6, 2, 0, 0],
                    [0, 0, 0, 8, 0, 0, 6, 4, 9]
                ],
                [
                    [5, 7, 9, 0, 0, 2, 8, 0, 0],
                    [2, 0, 0, 8, 0, 0, 7, 0, 9],
                    [0, 0, 8, 0, 0, 0, 2, 6, 0],
                    [0, 0, 7, 0, 0, 0, 0, 0, 6],
                    [0, 5, 4, 0, 9, 6, 0, 0, 0],
                    [0, 1, 0, 0, 7, 0, 0, 0, 0],
                    [0, 6, 0, 7, 2, 0, 4, 0, 0],
                    [0, 0, 3, 6, 0, 0, 0, 0, 7],
                    [9, 0, 2, 0, 0, 0, 3, 5, 0]
                ]
            ]
        }

        puzzle = puzzles[level][puzzle_number]
        for i in range(9):
            for j in range(9):
                if puzzle[i][j] != 0:
                    self.cells[i][j].insert(0, puzzle[i][j])
                    self.cells[i][j].config(state="disabled")
                else:
                    self.cells[i][j].delete(0, tk.END)
                    self.cells[i][j].config(state="normal")


    def reset(self):
        for i in range(9):
            for j in range(9):
                self.cells[i][j].config(state="normal")
                self.cells[i][j].delete(0, tk.END)
                
        self.start_time = time.time()

    def solve_backtracking(self):
        def is_valid_move(board, row, col, num):
            # Check if the number is already used in the row
            if num in board[row]:
                return False

            # Check if the number is already used in the column
            if num in [board[i][col] for i in range(9)]:
                return False

            # Check if the number is already used in the 3x3 grid
            start_row, start_col = 3 * (row // 3), 3 * (col // 3)
            for i in range(3):
                for j in range(3):
                    if board[start_row + i][start_col + j] == num:
                        return False
            return True

        def solve(board):
            for i in range(9):
                for j in range(9):
                    if board[i][j] == 0:
                        for num in range(1, 10):
                            if is_valid_move(board, i, j, num):
                                board[i][j] = num
                                if solve(board):
                                    return True
                                board[i][j] = 0
                        return False
            return True

        current_board = [[int(cell.get()) if cell.get() else 0 for cell in row] for row in self.cells]
        if solve(current_board):
            for i in range(9):
                for j in range(9):
                    self.cells[i][j].delete(0, tk.END)
                    self.cells[i][j].insert(0, current_board[i][j])
                    self.cells[i][j].config(state="disabled")
            elapsed_time = time.time() - self.start_time
            self.timer_label.config(text="Time: {:.3f} seconds".format(elapsed_time))  # Update timer label
            messagebox.showinfo("Solved", "Puzzle has been solved!")
        else:
            messagebox.showinfo("No Solution", "No solution exists for the current puzzle.")

    def solve_arc3(self):
        def is_valid_move(board, row, col, num):
            # Check if the number is already used in the row
            if num in board[row]:
                return False

            # Check if the number is already used in the column
            if num in [board[i][col] for i in range(9)]:
                return False

            # Check if the number is already used in the 3x3 grid
            start_row, start_col = 3 * (row // 3), 3 * (col // 3)
            for i in range(3):
                for j in range(3):
                    if board[start_row + i][start_col + j] == num:
                        return False
            return True

        def solve(board):
            for i in range(9):
                for j in range(9):
                    if board[i][j] == 0:
                        for num in range(1, 10):
                            if is_valid_move(board, i, j, num):
                                board[i][j] = num
                                if solve(board):
                                    return True
                                board[i][j] = 0
                        return False
            return True

        current_board = [[int(cell.get()) if cell.get() else 0 for cell in row] for row in self.cells]

        # Check for any cells with only one possible value and fill them in
        for i in range(9):
            for j in range(9):
                if current_board[i][j] == 0:
                    possible_values = [num for num in range(1, 10) if is_valid_move(current_board, i, j, num)]
                    if len(possible_values) == 1:
                        current_board[i][j] = possible_values[0]

        if solve(current_board):
            for i in range(9):
                for j in range(9):
                    self.cells[i][j].delete(0, tk.END)
                    self.cells[i][j].insert(0, current_board[i][j])
                    self.cells[i][j].config(state="disabled")
            elapsed_time = time.time() - self.start_time
            self.timer_label.config(text="Time: {:.3f} seconds".format(elapsed_time))  # Update timer label
            messagebox.showinfo("Solved", "Puzzle has been solved!")
        else:
            messagebox.showinfo("No Solution", "No solution exists for the current puzzle.")



    def update_timer(self):
        if self.start_time:
            elapsed_time = time.time() - self.start_time
            self.timer_label.config(text="Time: {:.3f} seconds".format(elapsed_time))
        self.master.after(100, self.update_timer)

root = tk.Tk()
app = SudokuGUI(root)
root.after(100, app.update_timer)
root.mainloop()